# Scenario Studio – Fuel Bus Demo
Run PF with explicit fuel buses. Adjust the YAMLs to change topology/params.


In [1]:
from energis.run.orchestrator import run_all
res = run_all()
res

ModuleNotFoundError: No module named 'energis'